In [792]:
import numpy as np
import pandas as pd
import pickle as pkl
import requests
import random
import time
import os
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup

In [164]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [760]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://www.amazon.com/s/gp/search/ref=sr_nr_p_n_feature_eleven_b_3?fst=as%3Aoff&rh=n%3A2335752011%2Cn%3A7072561011%2Ck%3Asmartphones%2Cp_n_feature_eleven_browse-bin%3A14674898011%5Cc14674900011%5Cc14674893011&keywords=smartphones&ie=UTF8&qid=1531354852&rnid=14674879011')
html_source = driver.page_source


In [212]:
# export html_source to local

with open('project2_html_src', 'w') as file:
    file.write(html_source)
    file.write(html1_source)
    file.write(str(more_html_source))
    file.close()

In [184]:
# click next page
driver.find_element_by_id('pagnNextString').click()

In [193]:
# get the page/html source from the rest of the pages
more_html_source = []

while True:
    driver.find_element_by_id('pagnNextString').click()
    time.sleep(random.choice(list(range(0,11))))
    more_html_source.append(driver.page_source)
    time.sleep(random.choice(list(range(0,11))))
    
    if 'srSprite lastPageRightArrow' in more_html_source:
        driver.close()
    else:
        continue

KeyboardInterrupt: 

### Need to compile all product links from the following:
* html_source
* html1_source
* more_html_source

In [222]:
soup = BeautifulSoup(html_source, 'html5lib')
prod_links = soup.find_all('a', class_ = 'a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal')

link_list = []
for link in prod_links:
    link_list.append(f'{link.get("href")}')


24

In [230]:
soup1 = BeautifulSoup(html1_source, 'html5lib')

prod_links_1 = soup.find_all('a', class_ = 'a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal')

for link in prod_links_1:
    link_list.append(f'{link.get("href")}')


72

In [245]:
# convert list to string to be able to use BeautifulSoup
last_html_source = ''.join(more_html_source)

soup_final = BeautifulSoup(last_html_source, 'html5lib')
prod_links_final = soup_final.find_all('a', class_ = 'a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal')

for link in prod_links_final:
    link_list.append(f'{link.get("href")}')


932

## Use Selenium to go through all obtained links to obtain data

In [ ]:
# Create empty lists for Selenium/BeautifulSoup to collect data in
rating = []
product_name = []
price = []
screensize = []
weight = []

d_memory = []
d_ram = []
d_camera = []
d_battery = []

In [771]:
# Scrape Amazon

driver = webdriver.Chrome(chromedriver)

for link in link_list[0:932]:
    time.sleep(random.choice(range(0,6)))
    driver.get(link)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html5lib')

    name = soup.find('span', id = 'productTitle', class_ = 'a-size-large')
    if str(name) == '[]':
        product_name.append('na')
    else:
        product_name.append(name.text.strip('\n '))
 

    price_origin = soup.find_all('span', id = 'priceblock_ourprice', class_ = 'a-size-medium a-color-price')
    price_alt = soup.find_all('td', class_ = 'comparison_baseitem_column')
    price_alt1 = soup.find_all('span', class_ = 'a-size-base a-color-price offer-price a-text-normal')
    if str(price_origin) == '[]' and str(price_alt) == '[]' and str(price_alt1) == '[]':
        price.append('na')
    if str(price_origin) != '[]':
        price.append(price_origin[0].text)
    else:
        pass
    if str(price_alt) != '[]' and str(price_origin) == '[]':
        price.append(price_alt[1].text.strip('\n ').strip('From '))
    else:
        pass
    if str(price_alt1) != '[]' and str(price_alt) == '[]' and str(price_origin) == '[]':
        price.append(price_alt1[0].text)
    else:
        pass
        

    scr_size = soup.find_all('td', class_ = 'comparison_baseitem_column')
    if str(scr_size) == '[]':
        screensize.append('na')
    else:
        screensize.append(scr_size[5].text.strip('\n '))

    wt = soup.find_all('td', class_ = 'comparison_baseitem_column')
    if str(wt) == '[]':
        weight.append('na')
    else:
        weight.append(wt[7].text.strip('\n '))

    ratings = soup.find_all('span', class_ = 'arp-rating-out-of-text')
    if str(ratings) == '[]':
        rating.append('na')
    else:
        rating.append(ratings[0].text)


In [794]:
# Compile data into DataFrame

df = pd.DataFrame(columns = ['product_name','rating','price','screensize','weight'])

df['product_name'] = product_name
df['rating'] = rating
df['price'] = price
df['screensize'] = screensize
df['weight'] = weight

df.head(10)

,product_name,rating,price,screensize,weight
0,BlackBerry Leap 16GB Factory Unlocked GSM 4G L...,3.2 out of 5 stars,$79.72,5 in,6 ounces
1,LG G3 D850 32GB Carrier Unlocked GSM Smartphon...,2.6 out of 5 stars,$127.53,na,na
2,Kyocera DuraForce E6560 16GB Unlocked GSM 4G L...,3.4 out of 5 stars,$79.95,4.5 in,7.05 ounces
3,LG G2 D800 GSM 4G LTE Unlocked Smartphone with...,3.7 out of 5 stars,$199.99,5.2 in,5.04 ounces
4,Samsung Galaxy Alpha G850a 32GB Carrier Unlock...,3.0 out of 5 stars,$84.99,na,na
5,HUAWEI Ascend Mate 2 16GB Unlocked GSM 6.1” Di...,4.1 out of 5 stars,$199.99,6.1 in,7.13 ounces
6,"LG G3, Metallic Black 32GB (Verizon Wireless)",3.3 out of 5 stars,na,na,na
7,Samsung Galaxy S5 Active G870a 16GB Unlocked G...,3.3 out of 5 stars,$189.99,5.1 in,6 ounces
8,Samsung Galaxy S6 G920P 32GB Black Boost Mobil...,3.6 out of 5 stars,$164.00,5.1 in,—
9,HTC One M8 Factory Unlocked Smartphone with 32...,3.3 out of 5 stars,$199.00,5 in,5.64 ounces


In [795]:
# Pickle and save all data gathered

with open('amazon_scrape_data.pkl', 'wb') as picklefile:
    pkl.dump(df, picklefile)